In [1]:
import os
import pandas as pd
import requests
import pathlib
from requests.auth import HTTPBasicAuth

api_key = os.environ.get("NCBI_API_KEY")

In [2]:
# Input
taxon_file = "../data/imported/taxids.list"

# Output
output_dir = pathlib.Path("../data/generated/download_genomes")
pathlib.Path.mkdir(output_dir, exist_ok=True)
genomes_metadata_file = output_dir / "genomes_metadata.csv"
genome_accessions_file = output_dir / "genome_accessions.list"
genomes_zip_file = output_dir / "genomes.zip"

In [3]:
def get_genome_dataset_report(taxon_id, api_key, params=None):
    """
    Get the genome dataset report for a given taxon id using NCBI API.
    Default parameters are:
    - reference_only=true
    - assembly_source=refseq
    - exclude_atypical=true # => don't include genomes that have assembly issues or are otherwise atypical
    - assembly_version=current
    """
    if params is None:
        params = {
            "filters.reference_only": "true",
            "filters.assembly_source": "refseq",
            "filters.exclude_atypical": "true",
            "filters.assembly_version": "current",
        }
    auth = HTTPBasicAuth("api-key", api_key)
    url = f"https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/taxon/{taxon_id}/dataset_report"
    response = requests.get(url=url, auth=auth, params=params)
    if "reports" not in response.json():
        print(f"No reports found for taxon id {taxon_id}")
        return
    return pd.DataFrame(
        _parse_genome_dataset_report_response(response.json(), taxon_id)
    )


def _parse_genome_dataset_report_response(response, query_taxid):
    for report in response["reports"]:
        yield {
            "query_taxid": query_taxid,
            "genome_accession": report["current_accession"],
            "genome_name": report["organism"]["organism_name"],
            "genome_taxid": report["organism"]["tax_id"],
            "assembly_level": report["assembly_info"]["assembly_level"],
            "assembly_status": report["assembly_info"]["assembly_status"],
            "assembly_name": report["assembly_info"]["assembly_name"],
            "refseq_category": report["assembly_info"]["refseq_category"]
            if "refseq_category" in report["assembly_info"]
            else pd.NA,
            "number_of_contigs": report["assembly_stats"]["number_of_contigs"],
            "contig_n50": report["assembly_stats"]["contig_n50"],
            "contig_l50": report["assembly_stats"]["contig_l50"],
            "genome_coverage": float(
                str(report["assembly_stats"]["genome_coverage"]).replace("x", "")
            )
            if "genome_coverage" in report["assembly_stats"]
            else pd.NA,
            "total_sequence_length": report["assembly_stats"]["total_sequence_length"],
            "total_ungapped_length": report["assembly_stats"]["total_ungapped_length"],
        }

In [4]:
refseq_params = {
    "filters.reference_only": "true",
    "filters.assembly_source": "refseq",
    "filters.exclude_atypical": "true",
    "filters.assembly_version": "current",
}

genome_dataset_report = get_genome_dataset_report("562", api_key, params=refseq_params)
genome_dataset_report

,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length
0,562,GCF_000005845.2,Escherichia coli str. K-12 substr. MG1655,511145,Complete Genome,current,ASM584v2,reference genome,1,4641652,1,<NA>,4641652,4641652
1,562,GCF_000008865.2,Escherichia coli O157:H7 str. Sakai,386585,Complete Genome,current,ASM886v2,reference genome,3,5498578,1,<NA>,5594605,5594605


In [5]:
def read_taxon_list(taxon_file):
    """
    Read a list of taxon ids from a file.
    """
    taxon_list = []
    with open(taxon_file) as f:
        taxon_list.extend(line.strip() for line in f if not line.startswith("#"))
    return sorted(set(taxon_list))


taxon_list = read_taxon_list(taxon_file)
print(f"{len(taxon_list)} taxon ids read from {taxon_file}")

162 taxon ids read from ../data/imported/taxids.list


In [6]:
refseq_genomes_dataset_reports = [
    get_genome_dataset_report(taxon_id, api_key, params=refseq_params)
    for taxon_id in taxon_list
]
refseq_genomes_df = pd.concat(refseq_genomes_dataset_reports).reset_index(drop=True)
refseq_genomes_df.sample()

No reports found for taxon id 1235797
No reports found for taxon id 142586
No reports found for taxon id 1454229
No reports found for taxon id 1686296
No reports found for taxon id 1849041
No reports found for taxon id 1935934
No reports found for taxon id 1955243
No reports found for taxon id 1965293
No reports found for taxon id 2039241
No reports found for taxon id 2040332
No reports found for taxon id 2049021
No reports found for taxon id 39491
No reports found for taxon id 39492
No reports found for taxon id 41978
No reports found for taxon id 885


,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length
129,626932,GCF_014163495.1,Alistipes indistinctus,626932,Complete Genome,current,ASM1416349v1,representative genome,2,3012737,1,196.0,3095585,3095585


In [7]:
refseq_genomes_df["refseq_category"].value_counts()

representative genome    147
Name: refseq_category, dtype: int64

In [8]:
refseq_genomes_df["assembly_level"].value_counts()

Complete Genome    70
Scaffold           43
Contig             25
Chromosome          9
Name: assembly_level, dtype: int64

In [9]:
failed_taxon_ids = list(
    set(taxon_list).difference(set(refseq_genomes_df["query_taxid"]))
)
print(f"{len(failed_taxon_ids)} taxon ids failed to retrieve genome dataset report")
failed_taxon_ids

15 taxon ids failed to retrieve genome dataset report


['1935934',
 '1965293',
 '1454229',
 '39491',
 '41978',
 '885',
 '2039241',
 '39492',
 '2040332',
 '1955243',
 '1235797',
 '142586',
 '1686296',
 '2049021',
 '1849041']

In [10]:
genbank_params = {
    "filters.assembly_source": "genbank",
    "filters.exclude_atypical": "true",
    "filters.assembly_version": "current",
    "table_fields": "assminfo-accession,assminfo-name",
    "page_size": 500,
}

genbank_genome_dataset_reports = [
    get_genome_dataset_report(failed_taxon_id, api_key, params=genbank_params)
    for failed_taxon_id in failed_taxon_ids
]
genbank_genomes_df = pd.concat(genbank_genome_dataset_reports).reset_index(drop=True)
genbank_genomes_df.sample(5)

No reports found for taxon id 1849041


,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length
893,39492,GCA_000210635.1,[Eubacterium] siraeum V10Sc8a,717961,Chromosome,current,ASM21063v1,<NA>,107,46087,18,<NA>,2836123,2779231
1241,142586,GCA_017633905.1,Eubacterium sp.,142586,Contig,current,ASM1763390v1,<NA>,166,12613,43,30.0,1754401,1754401
405,41978,GCA_017404665.1,Ruminococcus sp.,41978,Contig,current,ASM1740466v1,<NA>,79,49163,14,30.0,2465842,2465842
195,39491,GCA_022738285.1,[Eubacterium] rectale,39491,Contig,current,ASM2273828v1,<NA>,535,7235,109,434.4,2593202,2593202
1303,142586,GCA_021631215.1,Eubacterium sp.,142586,Scaffold,current,ASM2163121v1,<NA>,749,2885,201,8.4,1857763,1857493


In [11]:
genbank_genomes_df["assembly_level"].value_counts()

Contig             973
Scaffold           481
Chromosome           8
Complete Genome      3
Name: assembly_level, dtype: int64

In [12]:
def select_best_representative(df, consider_coverage=None):
    """
    Select the best representative genome from a genome dataset report.
    """
    if consider_coverage is None:
        consider_coverage = True
    taxa_dfs = []
    # convert columns to correct data types
    df = df.astype(
        {
            "query_taxid": "int",
            "genome_taxid": "int",
            "number_of_contigs": "int",
            "total_sequence_length": "int",
            "total_ungapped_length": "int",
            "contig_n50": "int",
            "contig_l50": "int",
            # "genome_coverage": "float",
        }
    )
    df["gaps"] = df["total_sequence_length"] - df["total_ungapped_length"]
    df["bases_per_contig"] = df["total_ungapped_length"] / df["number_of_contigs"]

    if consider_coverage:
        # if genome coverage is NA remove it from the dataframe
        df = df.dropna(subset=["genome_coverage"]).copy()

    # Select the representative genome with the highest genome coverage, least gaps and least contig_l50 and highest contig_n50.
    for _, grouped_taxa_df in df.groupby("query_taxid"):
        rows = grouped_taxa_df.shape[0]
        if rows == 1:
            taxa_dfs.append(grouped_taxa_df.iloc[0])
            continue

        # if a taxon has any 'Complete Genome's then shortlist those genomes to be considered for a representative genome.
        if any(grouped_taxa_df["assembly_level"] == "Complete Genome"):
            grouped_taxa_df = grouped_taxa_df.query(
                "assembly_level == 'Complete Genome'"
            ).copy()

        # Select the representative genome with the highest genome coverage, least gaps and least contig_l50 and highest contig_n50.
        grouped_taxa_df = grouped_taxa_df.sort_values(
            ["bases_per_contig", "contig_n50", "contig_l50", "genome_coverage", "gaps"],
            ascending=[False, False, True, False, True],
        )

        # append the best representative genome to the taxa_dfs list
        taxa_dfs.append(grouped_taxa_df.iloc[0])

    return pd.DataFrame(taxa_dfs)


select_best_representative(genbank_genomes_df).reset_index(drop=True).sample()

,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length,gaps,bases_per_contig
6,1454229,GCA_014385295.1,Bifidobacterium faecale,1454229,Contig,current,ASM1438529v1,<NA>,24,1422868,1,62.0,2257479,2257479,0,94061.625


In [13]:
genomes_df = pd.concat(
    [
        select_best_representative(refseq_genomes_df, consider_coverage=False),
        select_best_representative(genbank_genomes_df),
    ]
).reset_index(drop=True)
genomes_df.to_csv(genomes_metadata_file, index=False)
genomes_df.sample(5)

,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length,gaps,bases_per_contig
153,1454229,GCA_014385295.1,Bifidobacterium faecale,1454229,Contig,current,ASM1438529v1,<NA>,24,1422868,1,62.0,2257479,2257479,0,9.406162e+04
35,1735,GCF_003464225.1,Holdemanella biformis,1735,Scaffold,current,ASM346422v1,representative genome,87,63083,12,100.0,2477128,2476939,189,2.847056e+04
65,47678,GCF_018292205.1,Bacteroides caccae CL03T12C61,997873,Complete Genome,current,ASM1829220v1,representative genome,4,5485670,1,164.0,5499725,5499725,0,1.374931e+06
158,2039241,GCA_020024265.1,Anaerotignum sp.,2039241,Contig,current,ASM2002426v1,<NA>,50,62090,10,66.4,1877045,1877045,0,3.754090e+04
54,39486,GCF_000169235.1,Dorea formicigenerans ATCC 27755,411461,Contig,current,ASM16923v1,representative genome,16,318591,3,<NA>,3186031,3186031,0,1.991269e+05


In [14]:
genomes_df["genome_accession"].value_counts()

GCF_016889925.1    1
GCF_009721605.1    1
GCF_020341675.1    1
GCF_025149125.1    1
GCF_008571395.1    1
                  ..
GCF_018140635.1    1
GCF_003475105.1    1
GCF_902375065.1    1
GCF_900637515.1    1
GCA_028727085.1    1
Name: genome_accession, Length: 161, dtype: int64

In [15]:
## Check to make sure there are no duplicate genome accessions
assert all(
    genomes_df["genome_accession"].value_counts() == 1
), "Duplicate genome accessions found!"

In [16]:
# Save the genome accessions to a file
genomes_df["genome_accession"].to_csv(genome_accessions_file, index=False, header=False)

Manually added `GCF_013304735.1` and `GCF_013305105.1` to accessions list as substitutes for the two Blautia lutia genomes (Blautia sp. (95% luti) and Blautia sp. (97% luti))

No genomes could be found for NCBI TaxID `1849041`, but we have it assembled here: `s3://genomics-workflow-core/Results/HybridAssembly/MITI-MCB/SH0001499-00107/UNICYCLER/assembly.fasta`
So I'll add this to the list of genomes to be processed.

## Download the genomes

In [16]:
%%bash -s "$api_key" "$genome_accessions_file" "$genomes_zip_file"

datasets download genome accession \
    --api-key $1 \
    --inputfile $2 \
    --no-progressbar \
    --filename $3
    